# EDS 220 Fall 2021

## Visualizing global precipitation using Google Earth Engine

Let's get started with some data visualization and processing! Our tool for this exercise will be the Google Earth Engine (GEE):
https://earthengine.google.com/

GEE is a really powerful platform that allows easy access to many datasets without the need for a lot of pre-processing, so it's a good way to get our hands dirty with some basic mapping skills. Here we'll use the Python API, which will let us code in 'familiar' Python commands while still interfacing with the GEE platform. For this, we'll need to import the 'geemap' package in Python. 

The code chunk below checks to see whether geemap is already installed; if it's not, then it does the installation for us!

In [124]:
import subprocess

try:
    import geemap
except ImportError:
    print('Installing geemap ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])


Now we import the Google Earth Engine. Note: this will fail unless you activate the "ee" environment using conda before launching JupyterLab, otherwise Python won't know what "ee" means below.

We'll also import the pandas package to allow for manpulation of data subsets later on! It's good practice to do all of the importing of needed packages in one place (preferably at the start of your code) so that you can easily see what tools are available.

In [125]:
import ee
import geemap
import pandas as pd

## Create an interactive map

Now let's load the package that will allow us to map a given dataset. The default basemap is, unsurprisingly, Google Maps. Executing the code chunk below will display an empty Google Map that you can manipulate just like a regular Google Maps interface:

In [126]:
Map = geemap.Map(center=[40,-100], zoom=2)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

Now we need to tell GEE which dataset to layer on top of the base map. 

The next chunk of code will load some data; when executed, it will display within the map window above.

## Load data and plot over the basemap

The dataset we'll be using here is the ERA5 "reanalysis": essentially, a reanalysis is a numerical model (in this case, a weather model) that's been forced to match observations of the real world (in this case, atmospheric weather observations) as closely as possible. ERA5 is produced by the European Centre for Medium-Range Weather Forecasts, or ECMWF; so you'll see below that the parent directory for this dataset is called ECMWF. A description of the dataset is here:
https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_DAILY

The below chunk of code loads in the "mean_2m_air_temperature" and "total precipitation" fields, which are basically what they sound like: the temperature of air at an altitude of 2 meters, and the daily total precipitation, everywhere on Earth. Here we'll be working just with precipitation, but I've included temperature as well so you can see that other variables are just as easy to grab!

Both temperature and precipitation are stored as GEE "Image Collections"; there will be one image for every time value. This becomes important during later filtering and processing steps.

In [127]:
# Add Earth Engine dataset

gdat = ee.ImageCollection('ECMWF/ERA5/DAILY')
ts = gdat.select('mean_2m_air_temperature')

pr=gdat.select('total_precipitation')

This data isn't very mappable right now, since it contains information every day from 1979 to 2020. We need to filter it down to a single field for a time of our choice. Let's select an arbitrary time range: here, December of 2019.

In [128]:
tsdflt=ts.filter(ee.Filter.date('2019-12-01', '2019-12-02')).mean();
prdflt=pr.filter(ee.Filter.date('2019-12-01', '2019-12-02')).mean();


Now let's tell the map where to center itself! This is done with the "setCenter" command applied to the Map object. The first two arguments below are the longitude and latitude, respectively; the third is the zoom level for the map (smaller numbers = bigger maps).

In [129]:
# Map.setCenter(-90.162, 29.8597, 4)   # New Orleans, USA
# Map.setCenter(-114.9774, 31.9254, 4) # Mouth of the Colorado River, Mexico
# Map.setCenter(-111.1871, 37.0963, 3) # Lake Powell, USA
# Map.setCenter(149.412, -35.0789, 4)  # Lake George, Australia
Map.setCenter(105.26, 11.2134, 4)     # Mekong River Basin, SouthEast Asia
# Map.setCenter(90.6743, 22.7382, 4)   # Meghna River, Bangladesh
# Map.setCenter(81.2714, 16.5079, 4)   # Godavari River Basin Irrigation Project, India
# Map.setCenter(14.7035, 52.0985, 4)   # River Oder, Germany & Poland
# Map.setCenter(-59.1696, -33.8111, 4)  # Buenos Aires, Argentina\
#Map.setCenter(-74.4557, -8.4289, 4)  # Ucayali River, Peru


Now we can set a color palette to use when plotting the data layer. The following are the palettes specified for temperature and precipitation in the GEE description page for ERA5. Note that you also have to specify a max/min value for each of the variables!

In [130]:
VIS_TS = {
    'min':273,
    'max':310,
    'palette': ['#000080', '#0000D9', '#4000FF', '#8000FF', '#0080FF', '#00FFFF', '#00FF80',
    '#80FF00', '#DAFF00', '#FFFF00', '#FFF500', '#FFDA00', '#FFB000', '#FFA400',
    '#FF4F00', '#FF2500', '#FF0A00', '#FF00FF']
}

VIS_PREC = {
    'min':0,
    'max':0.1,
    'palette': ['#FFFFFF', '#00FFFF', '#0080FF', '#DA00FF', '#FFA400', '#FF0000']
}

In [131]:
Map.addLayer(prdflt, VIS_PREC,'total precipitation',opacity=0.3)

## Play with the map!

Now that everything has been set up, try playing around with things a bit. 

#### 1) Look at time variations

Change the date range in the ee.Filter.date command above, then replot the map layer. How different do things look? 

#### 2) Change map locations

Try zooming in and out on different parts of the map. What types of features do you notice? Are there particular patterns that seem to persist in particular places? (for example: check out the equatorial Pacific!)

## Look for historical storm events

Test out your map manipulation skills! Try filtering the data in time and space to locate a couple of particularly strong precipitation events:

- Montecito, California: January 9, 2018

- Rio Grande do Sul, Brazil: March 11-13, 2017

- Mumbai, India: July 26, 2005

(hint: Google is your friend for finding lats/lons for these places!)

Can you see any coherent patterns of precipitation in those locations on those days?


## Save images of your maps

Finally, save your work! There are a few different options for saving images from GEE; here we'll use the getThumbURL() method. This generates a JPG or PNG image, for which the URL is provided, then you can download the result. 

In [133]:
url = prdflt.getThumbUrl({
    'min': 0, 'max': 0.1, 'dimensions': 512, 
    'palette': ['#FFFFFF', '#00FFFF', '#0080FF', '#DA00FF', '#FFA400', '#FF0000']})
print(url)


https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/8f8a3756307e27243eb597246cf8a938-e460ce6e021424e0dbf8b7982d8b997c:getPixels


Once you've executed the above code, you can save it to whatever filename you like!